## Import libraries

In [4]:
import ee
import geemap

## Create an interactive map

In [5]:
Map = geemap.Map(center=[40, -100], zoom=4)

## Add Earth Engine Python script

In [6]:
# Add Earth Engine dataset
image = ee.Image("USGS/SRTMGL1_003")

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Chapter:      F4.0 Filter, Map, Reduce
#  Checkpoint:   F40c
#  Author:       Jeff Cardille
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

imgCol = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
# How many Tier 1 Landsat 5 images have ever been collected?
print("All images ever: ", imgCol.size().getInfo())

# How many images were collected in the 2000s?
startDate = '2000-01-01'
endDate = '2010-01-01'

imgColfilteredByDate = imgCol.filterDate(startDate, endDate)
print("All images 2000-2010: ", imgColfilteredByDate.size().getInfo())
# A smaller (but still large) number

ShanghaiImage = ee.Image(
'LANDSAT/LT05/C02/T1_L2/LT05_118038_20000606')
Map.centerObject(ShanghaiImage, 9)

imgColfilteredByDateHere = imgColfilteredByDate.filterBounds(ee.Geometry.Point([Map.get_center()[1], Map.get_center()[0]]))
print("All images here, 2000-2010: ", imgColfilteredByDateHere \
.size().getInfo())

L5FilteredLowCloudImages = imgColfilteredByDateHere \
.filterMetadata('CLOUD_COVER', 'less_than', 50)
print("Less than 50% clouds in this area, 2000-2010",
L5FilteredLowCloudImages.size().getInfo())

chainedFilteredSet = imgCol.filterDate(startDate, endDate) \
.filterBounds(ee.Geometry.Point([Map.get_center()[1], Map.get_center()[0]])) \
.filterMetadata('CLOUD_COVER', 'less_than', 50)
print('Chained: Less than 50% clouds in this area, 2000-2010',
chainedFilteredSet.size().getInfo())

efficientFilteredSet = imgCol.filterBounds(ee.Geometry.Point([Map.get_center()[1], Map.get_center()[0]])) \
.filterDate(startDate, endDate) \
.filterMetadata('CLOUD_COVER', 'less_than', 50)
print('Efficient filtering: Less than 50% clouds in this area, 2000-2010',
efficientFilteredSet.size().getInfo())

#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------


def func_mrg(oneL5Image):
    # compute the EVI for any Landsat 5 image. Note it's specific to
    # Landsat 5 images due to the band numbers. Don't run this exact
    # function for images from sensors other than Landsat 5.

    # Extract the bands and divide by 1e4 to account for scaling done.
    nirScaled = oneL5Image.select('SR_B4').divide(10000)
    redScaled = oneL5Image.select('SR_B3').divide(10000)
    blueScaled = oneL5Image.select('SR_B1').divide(10000)

    # Calculate the numerator, note that order goes from left to right.
    numeratorEVI = (nirScaled.subtract(redScaled)).multiply(
    2.5)

    # Calculate the denominator
    denomClause1 = redScaled.multiply(6)
    denomClause2 = blueScaled.multiply(7.5)
    denominatorEVI = nirScaled.add(denomClause1).subtract(
    denomClause2).add(1)

    # Calculate EVI and name it.
    landsat5EVI = numeratorEVI.divide(denominatorEVI).rename(
    'EVI')
    return (landsat5EVI)

makeLandsat5EVI = func_mrg

























L5EVIimages = efficientFilteredSet.map(makeLandsat5EVI)
print('Verifying that the .map gives back the same number of images: ',
L5EVIimages.size().getInfo())
print(L5EVIimages.getInfo())

Map.addLayer(L5EVIimages, {}, 'L5EVIimages', 1, 1)

#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------

L5EVImean = L5EVIimages.reduce(ee.Reducer.mean())
print(L5EVImean.getInfo())
Map.addLayer(L5EVImean, {
    'min': -1,
    'max': 2,
    'palette': ['red', 'white', 'green']
}, 'Mean EVI')

L5EVImedian = L5EVIimages.reduce(ee.Reducer.median())
print(L5EVImedian.getInfo())
Map.addLayer(L5EVImedian, {
    'min': -1,
    'max': 2,
    'palette': ['red', 'white', 'green']
}, 'Median EVI')

#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------

All images ever:  1869015
All images 2000-2010:  646696
All images here, 2000-2010:  125
Less than 50% clouds in this area, 2000-2010 83
Chained: Less than 50% clouds in this area, 2000-2010 83
Efficient filtering: Less than 50% clouds in this area, 2000-2010 83
Verifying that the .map gives back the same number of images:  83
{'type': 'ImageCollection', 'bands': [], 'version': 1723972040218094, 'id': 'LANDSAT/LT05/C02/T1_L2', 'properties': {'date_range': [441763200000, 1336176000000], 'visualization_2_bands': 'SR_B7,SR_B4,SR_B2', 'visualization_2_name': 'Shortwave Infrared (742)', 'period': 0, 'type_name': 'ImageCollection', 'keywords': ['cfmask', 'cloud', 'fmask', 'global', 'landsat', 'lasrc', 'lst', 'reflectance', 'sr', 'tm', 'usgs'], 'visualization_1_bands': 'SR_B4,SR_B3,SR_B2', 'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_SR_thumb.png', 'visualization_2_gain': '1.8,1.9,1.9', 'description': '<p>This dataset contains atmospherically corrected\nsurface reflectance and land 

## Display the interactive map

In [7]:
Map

Map(center=[31.754546511814873, 121.89145949319253], controls=(WidgetControl(options=['position', 'transparent…